# PaddleHub+HumanSeg实现视频人像抠图

> 使用PaddleHub内humanseg_serve模型进行视频流一键式人像抠图


**本项目将视频流内人体提取，并将背景替换成绿幕**


文档地址：[https://www.paddlepaddle.org.cn/hubdetail?name=humanseg_server&en_category=ImageSegmentation](https://www.paddlepaddle.org.cn/hubdetail?name=humanseg_server&en_category=ImageSegmentation)

In [ ]:
# 更新paddlehub版本至最新
! pip install paddlehub -U

In [ ]:
# 安装humanseg_server模型
! hub install humanseg_server

In [ ]:
# 查看已安装paddlehub模型
! hub list

In [ ]:
# 将待抠图视频上传至work文件夹内
# aistudio 最大上传的文件限制在150M以内
! cd ~/work && ls


In [ ]:
import cv2
import numpy as np
import paddlehub as hub
import os

os.chdir('/home/aistudio/work')
print(os.getcwd())
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

human_seg = hub.Module('humanseg_server')
cap_video = cv2.VideoCapture('cam_MP4.mp4')
fps = cap_video.get(cv2.CAP_PROP_FPS)
save_path = 'humanseg_server_video.mp4'
width = int(cap_video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap_video.get(cv2.CAP_PROP_FRAME_HEIGHT))
# cap_out = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, (width, height))
cap_out = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
prev_gray = None
prev_cfd = None

frame_toltal = cap_video.get(7)    # get all frame num
frame_index = 0                    # current frame index

while cap_video.isOpened():
    ret, frame_org = cap_video.read()
    if ret:
        [img_matting, prev_gray, prev_cfd] = human_seg.video_stream_segment(frame_org=frame_org, frame_id=cap_video.get(1), prev_gray=prev_gray, prev_cfd=prev_cfd, use_gpu=True)
        bg_im = cv2.imread('cam_1.jpg')
        bg_im = cv2.resize(bg_im, (np.shape(img_matting)[1],np.shape(img_matting)[0]))
        img_matting = np.repeat(img_matting[:, :, np.newaxis], 3, axis=2)
        #bg_im = np.ones_like(img_matting) * 0
        #print(img_matting)
        #bg_im[:, :, 1] = 255       # get green canva
        comb = (img_matting * frame_org + (1 - img_matting) * bg_im).astype(np.uint8)
        cap_out.write(comb)

        # show current information
        frame_index += 1
        print("Current Progress: {}/{}".format(frame_index, int(frame_toltal)))
    else:
        break

    # cv2.imshow('frame', comb)
    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #     break

cap_video.release()
cap_out.release()

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 